In [1]:
import numpy as np
import pandas as pd
import itertools
import re
import os
import difflib
import random
import requests
import seaborn as sns
import pickle
import operator
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import compete as com
import M_Arc_Dict.model_dict as model_dict

Using TensorFlow backend.
C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
import tensorflow as tf
print(tf.__version__)
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0
Num GPUs Available:  1


In [3]:
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Lambda, \
    Convolution2D,Convolution3D, Flatten, \
    Reshape, LSTM, Dropout, TimeDistributed, BatchNormalization,AveragePooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import time

class TimeHistory(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        

    
        
def generate_LSTM_model(input_shapes, output_shape,optimizer,rnn_layer_config,rnn_cell_config,dense_neuron_config,output_act, loss_fun,dropout, recurrent_dropout):

    model = Sequential()
    for i in range(rnn_layer_config):
        model.add(LSTM(units=rnn_cell_config, dropout=dropout, recurrent_dropout=recurrent_dropout,return_sequences=True, input_shape=input_shapes))    
    model.add(Flatten())
    for dense in dense_neuron_config:
        model.add(Dense(dense, activation=output_act))
    model.add(Dropout(dropout))
    model.add(Dense(output_shape))
    model.compile(loss=loss_fun, optimizer=optimizer)

    return model

def get_bi_LSTM_model(input_shapes, output_shape,optmizer,rnn_layer_config,rnn_cell_config,dense_neuron_config,output_act, loss_fun, dropout, recurrent_dropout):
    model = Sequential()
    for i in range(rnn_layer_config):
        if (i+1) == rnn_layer_config:
            model.add(Bidirectional(LSTM(units=rnn_cell_config, activation=output_act,dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True),merge_mode ='concat'))
        else:
            model.add(Bidirectional(LSTM(units=rnn_cell_config, activation=output_act,dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True), input_shape=input_shapes))    
   
    model.add(Flatten())
    
    for dense in dense_neuron_config:
        model.add(Dense(dense, activation=output_act))
        model.add(Dropout(dropout))
        
    model.add(Dense(output_shape))
    model.compile(optimizer=optimizer, loss=loss_fun)
    
    return model

In [4]:
# model = tf.keras.Sequential()
# # Add an Embedding layer expecting input vocab of size 1000, and
# # output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# # Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(128))

# # Add a Dense layer with 10 units.
# model.add(layers.Dense(10))

# model.summary()

In [5]:
# without time distributed
def generate_CNN_model(input_shape, output_shape, type_1_arr, type_2_arr, output_act, loss_tmp,
                       metrics=['loss']):
    filters = type_1_arr
    dense_layers = type_2_arr

    model = Sequential()

    if len(filters) == 1:
        model.add(Conv1D(filters=filters[0], kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    else:
        for fil_num in range(len(filters)):
            fil = filters[fil_num]
            if fil_num == 0:
                model.add(Conv1D(filters=filters[0], kernel_size=3, padding='same', activation='relu',
                                 input_shape=input_shape))
            else:
                model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())

    for dense in dense_layers:
        model.add(Dense(dense, activation=output_act))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, activation='relu'))
    model.compile(loss=loss_tmp, optimizer='adam', metrics=metrics)
    return model

In [6]:
def get_rand_rnn(min_num_layers,max_num_layers,min_num_cells,max_num_cells,layer_step_size,cell_step_size):
    total_layer = np.arange(min_num_layers, max_num_layers + 1, layer_step_size) #Generated total number of layers
    total_cells = np.arange(min_num_cells, max_num_cells + 1, cell_step_size) #Generated total number of cells or neurons

    #RNN block configuration
    rnn_layer_config = np.random.choice(total_layer) #Randomly generate rnn layers configuration
    rnn_cell_config = np.random.choice(total_cells) #Randomly generate rnn cells configuration

    #Dense block configuration
    dense_layer_config = np.random.choice(total_layer) #Randomly generate dense layers configuration
    dense_neuron_config = []
    
    for layer in range(dense_layer_config):
        tmp = np.random.choice(total_cells) #Randomly generate dense neurons configuration
        dense_neuron_config.append(tmp) #Adding each layer information realted to neurons
    return [rnn_layer_config, rnn_cell_config, dense_layer_config,dense_neuron_config]    


def construct_model(input_shapes,output_shape,optimizer,output_act,loss_fun,dropout,recurrent_dropout,model_info):
    model_type =model_info[1]
    if model_type ==  'mlstm00':
        rnn_layer_config = model_info[2][0]
        rnn_cell_config = model_info[2][1]
        dense_neuron_config = model_info[2][3]
        model = generate_LSTM_model(input_shapes, 
                                output_shape,
                                optimizer,
                                rnn_layer_config,
                                rnn_cell_config,
                                dense_neuron_config,
                                output_act, 
                                loss_fun,dropout, 
                                recurrent_dropout)
        
    elif model_type == 'mbilstm00':
        rnn_layer_config = model_info[2][0]
        rnn_cell_config = model_info[2][1]
        dense_neuron_config = model_info[2][3]   
        model = get_bi_LSTM_model(input_shapes, 
                                  output_shape,
                                  optimizer,
                                  rnn_layer_config,
                                  rnn_cell_config,
                                  dense_neuron_config,
                                  output_act, 
                                  loss_fun, 
                                  dropout, 
                                  recurrent_dropout)
        
    return model

In [7]:
#Constructing the dataset 

#Choose ahead and input window size for prediction
ahead = 1 # e.g., 1 day or 1 hours ahead
window_size = 3 # e.g, 3 days or 3 hours

# Choose the Input Features and Target 
features = ['cache_demand']
target = ['cache_demand']


# load dataset
dataset = pd.read_csv('MVNO_daily_cache/MVNO_daily_cache_2000.csv')

scaler = MinMaxScaler(feature_range=(0, 1))# normalize features (0 ~ 1)

for mvno in dataset['mvno'].unique():
    start = 0
    end = window_size
    num_samples = 0
    print("========",mvno,'=====')
    dataset_tmp =dataset[dataset['mvno']==mvno]
    dataset_tmp['cache_demand']=scaler.fit_transform(dataset_tmp[features])


    train_x = pd.DataFrame()
    train_y = pd.DataFrame()

    for i in range(len(dataset_tmp)-window_size):
        train_x = train_x.append(dataset_tmp.iloc[start:end],ignore_index = True)
        train_y = train_y.append(dataset_tmp.iloc[end],ignore_index = True)
        start = start+ahead
        end = end+ahead
        num_samples +=1

    # reshape the data    
    train_x = train_x[features].values.reshape(num_samples,window_size,len(features))
    train_y = train_y[target].values.reshape(num_samples,1)
        
    # Split the data into training and validation data   
    train_x =train_x[:int(len(train_x)*0.8)]
    train_y =train_y[:int(len(train_y)*0.8)]
    test_x =train_x[int(len(train_x)*0.8):]
    test_y =train_y[int(len(train_y)*0.8):]
    
    input_shapes = train_x.shape[1:3]
    output_shape = train_y.shape[1]

======== 0.0 =====


C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


======== 1.0 =====


C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


======== 2.0 =====


C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


======== 3.0 =====


C:\Users\netlab\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:

#Model configuration paramters
archi_table = 'marchi'
archi_list = ['mrnn']
model_name_list = ['mlstm00','mbilstm00']

min_num_layers = 1 # Minimun number of layers a.k.a hidden layers
max_num_layers = 20 # Maximun number of layers a.k.a hidden layers

min_num_cells = 1 # Minimun number of cells or neurons or perceptrons
max_num_cells = 256 # Maximun number of cells or neurons or perceptrons

layer_step_size = 1
cell_step_size = 1

output_act ='relu'
loss_fun = 'mse'
metrics = 'loss'
optimizer = 'adam'
dropout = 0.25
recurrent_dropout=0.1

#Objective
objective = 'min'

#Explore/Exploit parameters
threshold = [0.5]
alpha =0.1
gamma =0.5

#Training related parameters
Round =20
nr_epochs = 30
batch_size = 100
verbose =2

#locatings to stores q values
table_save_folder = 'q_tables/'


#initialized the population
P = 10


population = com.get_ini_model(P,
                  archi_list, 
                  model_name_list,
                  min_num_layers,
                  max_num_layers,
                  min_num_cells,
                  max_num_cells,
                  layer_step_size,
                  cell_step_size)


for round in range(Round):
    print('Round------', round)
    #Training
    for i in population:
        print("updated pop check", i)
        
    
    for i in range(len(population)):
        model_info = population[i]
        print(model_info)
        model = construct_model(input_shapes,
                        output_shape,
                        optimizer,
                        output_act,
                        loss_fun,
                        dropout,
                        recurrent_dropout,
                        model_info)

        time_callback = model_dict.TimeHistory()
        history = model.fit(train_x, train_y,
                        epochs=nr_epochs, batch_size=batch_size,
                        validation_data=(test_x, test_y),
                        verbose=verbose,callbacks=[time_callback])
        history_path ='logs/'
        file_name = history_path+model_info[1]
        history.history['time'] = time_callback.times
        
        obj = {str(model_info[2]):history.history}
        if not os.path.isfile(file_name+'.pickle'):
            com.q_store(file_name,obj)
        else:
            obj = com.q_load(file_name)
            obj[str(model_info[2])]=history.history
        com.q_store(file_name,obj)    
        
        file_name = history_path+'round00'
        obj = {round:{str(model_info[2]):history.history}}
        if not os.path.isfile(file_name+'.pickle'):
            com.q_store(file_name,obj)
        else:
            obj = com.q_load(file_name)
            obj[round]={str(model_info[2]):history.history}
        com.q_store(file_name,obj)   
        
        if len(population[i])!=4:
            population[i].append(np.mean(history.history['val_loss']))
        else:
            print('before loss', population[i][3])
            population[i][3] = np.mean(history.history['val_loss'])
            print('after loss', population[i][3])

    
    #Update q tables
    for model_info in population: 
        model_info_tmp = model_info[0:3]
        q=model_info[3]
        print('model_info',model_info)
        print("q", model_info[3])
        com.htable_update(objective,model_info_tmp,q,alpha,gamma,table_save_folder)  

    #Competition##
    win_list = np.arange(P).tolist()
    pop_index_arr = win_list
    compete_pairs =com.get_pairs(pop_index_arr)#get pairs
    population = com.competition(objective, 
                win_list, 
                population,
                threshold,
                min_num_layers,
                max_num_layers,
                min_num_cells,
                max_num_cells,
                layer_step_size,
                cell_step_size,
                archi_list,
                model_name_list)

Round------ 0
updated pop check ['mrnn', 'mbilstm00', [20, 194, 18, [185, 116, 100, 202, 134, 47, 136, 161, 167, 234, 188, 87, 217, 23, 194, 56, 139, 92]]]
updated pop check ['mrnn', 'mlstm00', [5, 194, 8, [146, 70, 6, 23, 173, 137, 185, 154]]]
updated pop check ['mrnn', 'mlstm00', [4, 183, 9, [49, 237, 18, 62, 206, 236, 77, 121, 215]]]
updated pop check ['mrnn', 'mlstm00', [1, 109, 19, [106, 12, 50, 122, 28, 87, 219, 101, 149, 69, 170, 13, 147, 45, 55, 9, 32, 59, 158]]]
updated pop check ['mrnn', 'mbilstm00', [3, 115, 1, [126]]]
updated pop check ['mrnn', 'mlstm00', [18, 69, 17, [18, 1, 236, 215, 143, 158, 7, 153, 225, 165, 66, 55, 91, 131, 72, 157, 82]]]
updated pop check ['mrnn', 'mbilstm00', [5, 18, 16, [111, 224, 50, 197, 236, 62, 80, 73, 79, 87, 137, 221, 21, 12, 149, 58]]]
updated pop check ['mrnn', 'mlstm00', [20, 30, 6, [164, 241, 44, 175, 78, 69]]]
updated pop check ['mrnn', 'mlstm00', [18, 231, 10, [128, 48, 34, 150, 240, 244, 94, 199, 129, 99]]]
updated pop check ['mrnn', '

Epoch 6/30
 - 0s - loss: 0.0144 - val_loss: 0.0113
Epoch 7/30
 - 0s - loss: 0.0167 - val_loss: 0.0105
Epoch 8/30
 - 0s - loss: 0.0186 - val_loss: 0.0106
Epoch 9/30
 - 0s - loss: 0.0168 - val_loss: 0.0115
Epoch 10/30
 - 0s - loss: 0.0156 - val_loss: 0.0142
Epoch 11/30
 - 0s - loss: 0.0153 - val_loss: 0.0171
Epoch 12/30
 - 0s - loss: 0.0162 - val_loss: 0.0161
Epoch 13/30
 - 0s - loss: 0.0151 - val_loss: 0.0133
Epoch 14/30
 - 0s - loss: 0.0147 - val_loss: 0.0117
Epoch 15/30
 - 0s - loss: 0.0146 - val_loss: 0.0112
Epoch 16/30
 - 0s - loss: 0.0145 - val_loss: 0.0115
Epoch 17/30
 - 0s - loss: 0.0155 - val_loss: 0.0123
Epoch 18/30
 - 0s - loss: 0.0142 - val_loss: 0.0126
Epoch 19/30
 - 0s - loss: 0.0135 - val_loss: 0.0117
Epoch 20/30
 - 0s - loss: 0.0144 - val_loss: 0.0107
Epoch 21/30
 - 0s - loss: 0.0144 - val_loss: 0.0104
Epoch 22/30
 - 0s - loss: 0.0134 - val_loss: 0.0107
Epoch 23/30
 - 0s - loss: 0.0135 - val_loss: 0.0108
Epoch 24/30
 - 0s - loss: 0.0133 - val_loss: 0.0103
Epoch 25/30
 - 0

Epoch 2/30
 - 0s - loss: 0.0679 - val_loss: 0.0322
Epoch 3/30
 - 0s - loss: 0.0589 - val_loss: 0.0262
Epoch 4/30
 - 0s - loss: 0.0500 - val_loss: 0.0207
Epoch 5/30
 - 0s - loss: 0.0410 - val_loss: 0.0162
Epoch 6/30
 - 0s - loss: 0.0340 - val_loss: 0.0130
Epoch 7/30
 - 0s - loss: 0.0270 - val_loss: 0.0113
Epoch 8/30
 - 0s - loss: 0.0239 - val_loss: 0.0114
Epoch 9/30
 - 0s - loss: 0.0205 - val_loss: 0.0132
Epoch 10/30
 - 0s - loss: 0.0179 - val_loss: 0.0161
Epoch 11/30
 - 0s - loss: 0.0195 - val_loss: 0.0189
Epoch 12/30
 - 0s - loss: 0.0193 - val_loss: 0.0205
Epoch 13/30
 - 0s - loss: 0.0201 - val_loss: 0.0205
Epoch 14/30
 - 0s - loss: 0.0195 - val_loss: 0.0191
Epoch 15/30
 - 0s - loss: 0.0201 - val_loss: 0.0175
Epoch 16/30
 - 0s - loss: 0.0185 - val_loss: 0.0160
Epoch 17/30
 - 0s - loss: 0.0187 - val_loss: 0.0147
Epoch 18/30
 - 0s - loss: 0.0185 - val_loss: 0.0139
Epoch 19/30
 - 0s - loss: 0.0203 - val_loss: 0.0134
Epoch 20/30
 - 0s - loss: 0.0183 - val_loss: 0.0132
Epoch 21/30
 - 0s - 

Epoch 23/30
 - 0s - loss: 0.0137 - val_loss: 0.0109
Epoch 24/30
 - 0s - loss: 0.0134 - val_loss: 0.0107
Epoch 25/30
 - 0s - loss: 0.0134 - val_loss: 0.0107
Epoch 26/30
 - 0s - loss: 0.0135 - val_loss: 0.0107
Epoch 27/30
 - 0s - loss: 0.0128 - val_loss: 0.0107
Epoch 28/30
 - 0s - loss: 0.0131 - val_loss: 0.0110
Epoch 29/30
 - 0s - loss: 0.0135 - val_loss: 0.0109
Epoch 30/30
 - 0s - loss: 0.0120 - val_loss: 0.0109
['mrnn', 'mbilstm00', [3, 18, 16, [111, 224, 50, 197, 236, 62, 80, 73, 79, 87, 137, 221, 21, 12, 149, 58]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 3s - loss: 0.0757 - val_loss: 0.0358
Epoch 2/30
 - 0s - loss: 0.0630 - val_loss: 0.0250
Epoch 3/30
 - 0s - loss: 0.0460 - val_loss: 0.0146
Epoch 4/30
 - 0s - loss: 0.0294 - val_loss: 0.0120
Epoch 5/30
 - 0s - loss: 0.0219 - val_loss: 0.0214
Epoch 6/30
 - 0s - loss: 0.0238 - val_loss: 0.0193
Epoch 7/30
 - 0s - loss: 0.0209 - val_loss: 0.0153
Epoch 8/30
 - 0s - loss: 0.0181 - val_loss: 0.0137
Epoch 9/30
 - 0s - loss

Epoch 19/30
 - 0s - loss: 0.0177 - val_loss: 0.0149
Epoch 20/30
 - 0s - loss: 0.0189 - val_loss: 0.0145
Epoch 21/30
 - 0s - loss: 0.0175 - val_loss: 0.0147
Epoch 22/30
 - 0s - loss: 0.0180 - val_loss: 0.0147
Epoch 23/30
 - 0s - loss: 0.0173 - val_loss: 0.0147
Epoch 24/30
 - 0s - loss: 0.0181 - val_loss: 0.0153
Epoch 25/30
 - 0s - loss: 0.0176 - val_loss: 0.0155
Epoch 26/30
 - 0s - loss: 0.0172 - val_loss: 0.0155
Epoch 27/30
 - 0s - loss: 0.0174 - val_loss: 0.0153
Epoch 28/30
 - 0s - loss: 0.0178 - val_loss: 0.0150
Epoch 29/30
 - 0s - loss: 0.0181 - val_loss: 0.0148
Epoch 30/30
 - 0s - loss: 0.0172 - val_loss: 0.0144
['mrnn', 'mlstm00', [6, 13, 19, [197, 66, 164, 150, 19, 122, 191, 162, 234, 125, 189, 71, 186, 159, 44, 230, 223, 187, 35]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 3s - loss: 0.0745 - val_loss: 0.0309
Epoch 2/30
 - 0s - loss: 0.0491 - val_loss: 0.0115
Epoch 3/30
 - 0s - loss: 0.0218 - val_loss: 0.0354
Epoch 4/30
 - 0s - loss: 0.0252 - val_loss: 0.0145
Ep

...

Epoch 30/30
 - 1s - loss: 0.0199 - val_loss: 0.0141
['mrnn', 'mbilstm00', [2, 82, 3, [66, 100, 115]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 2s - loss: 0.0731 - val_loss: 0.0303
Epoch 2/30
 - 0s - loss: 0.0513 - val_loss: 0.0156
Epoch 3/30
 - 0s - loss: 0.0272 - val_loss: 0.0121
Epoch 4/30
 - 0s - loss: 0.0214 - val_loss: 0.0287
Epoch 5/30
 - 0s - loss: 0.0243 - val_loss: 0.0205
Epoch 6/30
 - 0s - loss: 0.0181 - val_loss: 0.0129
Epoch 7/30
 - 0s - loss: 0.0165 - val_loss: 0.0111
Epoch 8/30
 - 0s - loss: 0.0173 - val_loss: 0.0108
Epoch 9/30
 - 0s - loss: 0.0178 - val_loss: 0.0111
Epoch 10/30
 - 0s - loss: 0.0176 - val_loss: 0.0122
Epoch 11/30
 - 0s - loss: 0.0160 - val_loss: 0.0143
Epoch 12/30
 - 0s - loss: 0.0166 - val_loss: 0.0159
Epoch 13/30
 - 0s - loss: 0.0154 - val_loss: 0.0152
Epoch 14/30
 - 0s - loss: 0.0166 - val_loss: 0.0134
Epoch 15/30
 - 0s - loss: 0.0156 - val_loss: 0.0123
Epoch 16/30
 - 0s - loss: 0.0149 - val_loss: 0.0115
Epoch 17/30
 - 0s - loss: 0.01

Epoch 26/30
 - 0s - loss: 0.0121 - val_loss: 0.0110
Epoch 27/30
 - 0s - loss: 0.0124 - val_loss: 0.0111
Epoch 28/30
 - 0s - loss: 0.0120 - val_loss: 0.0110
Epoch 29/30
 - 0s - loss: 0.0130 - val_loss: 0.0111
Epoch 30/30
 - 0s - loss: 0.0122 - val_loss: 0.0111
before loss 0.01471709068864584
after loss 0.012854574558635553
model_info ['mrnn', 'mbilstm00', [13, 203, 15, [207, 14, 211, 124, 253, 2, 188, 141, 105, 180, 27, 229, 52, 83, 204]], 0.015801001992076635]
q 0.015801001992076635
model_info ['mrnn', 'mlstm00', [3, 80, 7, [113, 192, 20, 4, 166, 73, 65]], 0.01628610317905744]
q 0.01628610317905744
model_info ['mrnn', 'mlstm00', [5, 126, 5, [52, 144, 255, 68, 222]], 0.016454208983729283]
q 0.016454208983729283
model_info ['mrnn', 'mlstm00', [14, 220, 17, [11, 72, 214, 190, 65, 216, 154, 145, 250, 222, 133, 150, 29, 101, 65, 118, 190]], 0.016319655627012254]
q 0.016319655627012254
model_info ['mrnn', 'mbilstm00', [13, 199, 1, [17]], 0.015785775830348334]
q 0.015785775830348334
model_inf

Epoch 26/30
 - 0s - loss: 0.0168 - val_loss: 0.0152
Epoch 27/30
 - 0s - loss: 0.0176 - val_loss: 0.0148
Epoch 28/30
 - 0s - loss: 0.0174 - val_loss: 0.0149
Epoch 29/30
 - 0s - loss: 0.0174 - val_loss: 0.0153
Epoch 30/30
 - 0s - loss: 0.0175 - val_loss: 0.0156
['mrnn', 'mbilstm00', [14, 116, 9, [245, 245, 255, 251, 77, 101, 210, 255, 178]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 16s - loss: 0.0706 - val_loss: 0.0221
Epoch 2/30
 - 1s - loss: 0.0347 - val_loss: 0.0282
Epoch 3/30
 - 1s - loss: 0.0252 - val_loss: 0.0159
Epoch 4/30
 - 1s - loss: 0.0180 - val_loss: 0.0115
Epoch 5/30
 - 1s - loss: 0.0188 - val_loss: 0.0120
Epoch 6/30
 - 1s - loss: 0.0178 - val_loss: 0.0152
Epoch 7/30
 - 1s - loss: 0.0182 - val_loss: 0.0189
Epoch 8/30
 - 1s - loss: 0.0181 - val_loss: 0.0166
Epoch 9/30
 - 1s - loss: 0.0174 - val_loss: 0.0142
Epoch 10/30
 - 1s - loss: 0.0177 - val_loss: 0.0138
Epoch 11/30
 - 1s - loss: 0.0177 - val_loss: 0.0141
Epoch 12/30
 - 1s - loss: 0.0171 - val_loss: 0.01

 - 0s - loss: 0.0170 - val_loss: 0.0159
Epoch 22/30
 - 0s - loss: 0.0174 - val_loss: 0.0156
Epoch 23/30
 - 0s - loss: 0.0173 - val_loss: 0.0153
Epoch 24/30
 - 0s - loss: 0.0168 - val_loss: 0.0150
Epoch 25/30
 - 0s - loss: 0.0174 - val_loss: 0.0156
Epoch 26/30
 - 0s - loss: 0.0176 - val_loss: 0.0148
Epoch 27/30
 - 0s - loss: 0.0174 - val_loss: 0.0153
Epoch 28/30
 - 0s - loss: 0.0176 - val_loss: 0.0153
Epoch 29/30
 - 0s - loss: 0.0173 - val_loss: 0.0149
Epoch 30/30
 - 0s - loss: 0.0171 - val_loss: 0.0158
['mrnn', 'mbilstm00', [7, 190, 17, [211, 8, 208, 102, 67, 201, 149, 243, 175, 41, 33, 168, 189, 76, 172, 99, 116]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 9s - loss: 0.0754 - val_loss: 0.0342
Epoch 2/30
 - 0s - loss: 0.0589 - val_loss: 0.0202
Epoch 3/30
 - 0s - loss: 0.0370 - val_loss: 0.0111
Epoch 4/30
 - 0s - loss: 0.0210 - val_loss: 0.0318
Epoch 5/30
 - 0s - loss: 0.0259 - val_loss: 0.0233
Epoch 6/30
 - 0s - loss: 0.0194 - val_loss: 0.0151
Epoch 7/30
 - 0s - loss: 

Epoch 21/30
 - 0s - loss: 0.0177 - val_loss: 0.0150
Epoch 22/30
 - 1s - loss: 0.0182 - val_loss: 0.0145
Epoch 23/30
 - 0s - loss: 0.0166 - val_loss: 0.0150
Epoch 24/30
 - 0s - loss: 0.0175 - val_loss: 0.0160
Epoch 25/30
 - 0s - loss: 0.0177 - val_loss: 0.0162
Epoch 26/30
 - 0s - loss: 0.0171 - val_loss: 0.0145
Epoch 27/30
 - 0s - loss: 0.0171 - val_loss: 0.0147
Epoch 28/30
 - 0s - loss: 0.0178 - val_loss: 0.0157
Epoch 29/30
 - 0s - loss: 0.0175 - val_loss: 0.0159
Epoch 30/30
 - 1s - loss: 0.0175 - val_loss: 0.0146
['mrnn', 'mbilstm00', [2, 185, 16, [254, 211, 73, 199, 83, 247, 149, 86, 129, 68, 203, 73, 203, 210, 77, 101]]]
Train on 290 samples, validate on 58 samples
Epoch 1/30
 - 3s - loss: 0.0754 - val_loss: 0.0338
Epoch 2/30
 - 0s - loss: 0.0579 - val_loss: 0.0185
Epoch 3/30
 - 0s - loss: 0.0320 - val_loss: 0.0139
Epoch 4/30
 - 0s - loss: 0.0256 - val_loss: 0.0272
Epoch 5/30
 - 0s - loss: 0.0219 - val_loss: 0.0156
Epoch 6/30
 - 0s - loss: 0.0203 - val_loss: 0.0122
Epoch 7/30
 - 0s 